In [70]:
import pandas as pd
import numpy as np
from langdetect import detect, detect_langs
import sys

In [32]:
lyrics = pd.read_csv('lyrics.csv')
lyrics.head()

,index,song,year,artist,genre,lyrics
0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\nYou know I'm gonna cu..."
1,1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy,\nit's like you see..."
2,2,honesty,2009,beyonce-knowles,Pop,If you search\nFor tenderness\nIt isn't hard t...
3,3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I\n[Verse 1:]\nIf I wrote..."
4,4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po..."


In [49]:
lyrics = lyrics[lyrics.lyrics.astype(str) != 'nan']

In [74]:
artists = lyrics.artist.unique()
filt = []
with open("artists.txt", "w") as f:
    for i, artist in enumerate(artists):
        sys.stdout.write('\r' + str(i).zfill(5))
        alyrics = lyrics[lyrics.artist == artist]
        nsongs = min(alyrics.shape[0], 10)
        songs = alyrics.sample(nsongs).lyrics
        en = 0
        for song in songs:
            try:
                if detect(song) == 'en':
                    en += 1
            except:
                pass
        if en/nsongs > 0.7:
            f.write(artist + '\n')
        else:
            filt.append(artist)

14542

In [77]:
with open("filtered.txt", "w") as fi:
    for artist in filt:
        fi.write(artist + '\n')    